In [ ]:
%load_ext autoreload
%autoreload 2

import csv
import ipywidgets as widgets
import json
import openai
import os
import pandas as pd
import papermill as pm
import scrapbook as sb

from IPython.display import display

from functions import gpt

from tqdm.notebook import tqdm_notebook
tqdm_notebook().pandas()

pd.set_option('display.max_colwidth', None)

In [ ]:
base_name = "2021_Hashimoto_Neural_ODE_and_holographic_QCD_PUB"
project_folder = "diygenomics-projects"
sub_category = "math"
work_bucket = "AdS-CFT"
external_id = "2023_05_22_92dc0613b4493d7b5847g"

In [ ]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket,
                                       base_name, 'mathpix', *args)

model = 'gpt-4' # 'gpt-3.5-turbo' # 'gpt-4'
index_col = 'uuid'

openai_model = 'text-embedding-ada-002'

input_file = 'extracted_annotated_math.csv'
output_file = 'extracted_annotated_math.csv'

math_column = 'clean_math'

In [ ]:
system_prompt = """You are an academic researcher and mathmatician in information systems biology.
- Convert the Latex expression into python using sympy: 
- Do not output anything except code.
- Do not provide an explaination.
- Do not include comments.
"""

In [ ]:
df = pd.read_csv(file_path(input_file), index_col=index_col)

In [ ]:
general_errors = {}

def generate_python_math(row):  
    response = None

    if len(row[math_column]) > 0:
        possible_response = gpt.chat_create(system_prompt, row[math_column], model, output_json=False)
        if 'general_error' in possible_response:
            general_errors[row.name] = response
        else:
            response = possible_response
        
    return response

In [ ]:
df['gpt_python_math'] = df.progress_apply(lambda row: generate_python_math(row), axis=1)

In [ ]:
df.to_csv(file_path(output_file), quoting=csv.QUOTE_MINIMAL)

In [ ]:
# convert this to python using sympy:

# \mathrm{d} s^{2}=-f(\eta) \mathrm{d} t^{2}+\mathrm{d} \eta^{2}+g(\eta)\left(\mathrm{d} x_{1}^{2}+\cdots+\mathrm{d} x_{d-1}^{2}\right)

# do not output anything except code. do not provide an explaination.